# Data Collection

In [2]:
#imports
from datetime import date
import pandas as pd
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium import webdriver # driving the actions, links up with the browser
from bs4 import BeautifulSoup
import time
import re
from pick import pick
import os

In [9]:
PATH =  r'C:\Users\esthe\Documents\Coding\chromedriver_win32\chromedriver.exe'

yes = ["Yes", "Y", "YES", "y", "yes"]
starting_page = "http://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"
output_name = 'WW_postings.csv'
choice = 'Shortlist'

In [10]:
df = pd.DataFrame(columns=['ID', 'Job Title', 'Organization', 'Division', 'Openings', 'Internal Status', 'City', 'Level',
                           'Applications', 'App Deadline', 'Work Term Duration', "Job Summary", "Job Responsibilities", "Required Skills", 
                           "Targeted Degrees and Disciplines", "Application Documents Required"])

def get_job_lists(choice, browser, output_name):
    # Navigate to the job listings page then wait for the javascript to load
    browser.find_element_by_link_text(choice).click()

    #input("Have you gotten to the page you want to start at?")
    
    # Get the HTML of the page and check if this is the last page (next_page_buttons)
    page_html = browser.execute_script("return document.body.innerHTML")
    pattern = r'<a href=".+?" onclick="loadPostingTable(.+?)">\s*»\s*<\/a>'
    next_page_buttons = []
    next_page_buttons = re.findall(pattern, page_html)

    counter_page = 0
    # Scrape the tables and save it to output_name as a CSV
    job_posting_importants_order = ['Work Term Duration:', "Job Summary:", "Job Responsibilities:", "Required Skills:", 
                                    "Targeted Degrees and Disciplines:", "Application Documents Required:"]
    with open(output_name, 'w') as f:
        row_count = 0
        #job_count = 0
        while (counter_page < 4): # We only want to do 4 pages of WaterlooWorks Postings
            counter_page += 1
            os.system('clear')

            # wait for JavaScript to load then download the page info
            time.sleep(2)
            soup = BeautifulSoup(page_html, "html.parser")
            
            for tr in soup.find_all('tr')[1:]: # iterating throw all rows
                #job_count += 1
                #print(job_count)
                try:
                    # all the data that you need to click into the posting to get
                    job_posting_importants = {"Work Term Duration:": None, # might need to do some regex filtering
                                             "Job Summary:" : None,
                                             "Job Responsibilities:": None,
                                             "Required Skills:" : None,
                                             "Targeted Degrees and Disciplines:": None,
                                             "Application Documents Required:": None} 

                    tds = tr.find_all('td') # iterating all cells in each row.
                    count = 0
                    job_info_list = []
                    for x in tds[0:-1]: # iterating through all columns in the row
                        if (count >= 2): # In this case, we have a colmun where we can directly save the column information
                            job_info_list.append(str(x.text))
                        if (count == 0): # this column contains the button we need se we can click into the job posting
                            link_container = x.find_all('a')[2]
                            onclick_link = link_container['onclick']

                            # clicking the onclick link:
                            browser.execute_script(onclick_link)
                            browser.switch_to.window(browser.window_handles[1])

                            # give the browser 1.5 seconds for the website to catch up with the code
                            browser.implicitly_wait(1.5)
                            time.sleep(2)

                            #saving the contents of the html of the new page:
                            indiv_html = browser.execute_script("return document.body.innerHTML")
                            indiv_posting = BeautifulSoup(indiv_html, "html.parser")

                            # finding the table in the page with the desired information about job summary and responsibilities
                            job_posting_information_table = indiv_posting.find_all('table', {'class': 'table-bordered'})[1]

                            # iterating through all rows in the table
                            for row in job_posting_information_table.find_all('tr'): 
                                cell_list = row.find_all('td')
                                try:
                                    key = cell_list[0].get_text().strip()
                                except:
                                    continue

                                if (key in job_posting_importants):
                                    try:
                                        job_posting_importants[key] = cell_list[1].get_text().strip()
                                    except:
                                        job_posting_importants[key] = ""

                            #finding the table in the page that contains application documents required
                            application_information_table = indiv_posting.find_all('table', {'class': 'table-bordered'})[2]
                            job_posting_importants["Application Documents Required:"] = application_information_table.find_all('td')[3].get_text().strip()

                            browser.close() # close current tab
                            browser.switch_to.window(browser.window_handles[0])
                            
                        count += 1
                    # Now add all info from the page we clicked into
                    for col in job_posting_importants_order:
                        job_info_list.append(job_posting_importants[col])

                    df.loc[row_count] = job_info_list
                    row_count+=1
                except:
                    continue
                
            # navigate to the next page and get the HTML, next page buttons
            try:
                next_page_buttons = re.findall(pattern, page_html)
                browser.find_element_by_link_text("»").click()
                time.sleep(2)
                page_html = browser.execute_script("return document.body.innerHTML")
            except:
                return


browser = webdriver.Chrome(PATH)
browser.get(starting_page)

input("Have you logged in to Waterloo Works and navigated to the 'Hire Waterloo Coops' Page?")

get_job_lists(choice, browser, output_name)

Have you logged in to Waterloo Works and navigated to the 'Hire Waterloo Coops' Page?y


In [41]:
df.head(3)

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,185577,\n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Planner\n\...,City of St Catharines,Municipal Office,1,Open for Applications,St. Catharines,"Intermediate, Senior",12,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Student Planner,To research assigned projects for data relevan...,Students should be experienced in virtual meet...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
1,186774,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,City of Waterloo,Development Services,1,Open for Applications,Waterloo,"Intermediate, Senior",63,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,The successful candidate will have involvement...,Growth Management Projects may include:\n\nLan...,The candidate should have a strong working kno...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
2,185901,\n\n\n\n\t\t\t\t\t\t\t\t\t\tFleet Assistant\n\...,City of Waterloo,Divisional Office,1,Open for Applications,Waterloo,Junior,9,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,A co-op opportunity exists for a student to su...,"Provide support to the Fleet Coordinator, Flee...",Currently enrolled in a relevant post-secondar...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."


In [71]:
# df = pd.read_csv('page6.csv')
# df = df.drop(['Unnamed: 0'], axis=1)

# Data Cleaning

In [62]:
'''
Things to Change
0. Get rid of whitespaces for job title - DONE
1. Get rid of capital NEW at the start of job titles - DONE
2. get rid of divison - DONE 
3. get rid of internal status - DONE
4. turn level into 3 binary columns - Dropped it, don't think its that important
5. filter all until only app deadline <= jan 19 - DONE
6. whitespace for targeted clusters - DONE
7. App docs required turns into need cover letter? - DONE
8. change Work Term Duration to [1,4]
9. whitepsace for app deadlines -DONE

'''
df = df.fillna("")

In [63]:
def surrounding_space_remover(str):
    return ' '.join(str.split())
    
df_space_remover = np.vectorize(surrounding_space_remover)

for column in ['Job Title', 'App Deadline', 'Targeted Degrees and Disciplines', "Job Summary", "Job Responsibilities", "Required Skills", "Targeted Degrees and Disciplines"]:
    df[column] = df_space_remover(df[column])

In [65]:
def new_removal(str):
    try:
        if (str[0:3] == "NEW"): 
            return str[3:]
    except:
        return str # not enough chars in job title
    return str # if no NEW in job title

df_new_remover = np.vectorize(new_removal)

df['Job Title'] = df_new_remover(df["Job Title"])

In [72]:
def day_extractor(str):
    str = str.replace(",", " ")
    str = [int(s) for s in str.split() if s.isdigit()]
    return str[0]

df_day_extractor = np.vectorize(day_extractor)

df['App Deadline'] = df_day_extractor(df["App Deadline"])
#adf = df[df["App Deadline"] <= 19]


In [67]:
def cover_letter_needed(mystr):
    return ("Cover Letter" in mystr)

df_cover_letter_needed = np.vectorize(cover_letter_needed)

df["Application Documents Required"] = df_cover_letter_needed(df["Application Documents Required"])
df.rename(columns={"Application Documents Required": "Cover Letter Needed"}, inplace=True)

In [68]:
def work_term_duration(str):
    if (str == '4 month work term'): return 1
    if (str == '8 month consecutive work term required'): return 2
    if (str == '8 month consecutive work term preferred'): return 3
    if (str == '2 work term commitment preferred'): return 4
    if (str == '2 work term commitment required'): return 5
    else: return 6 # in case theres smth weird
    
df_work_term_duration = np.vectorize(work_term_duration)

df["Work Term Duration"] = df_work_term_duration(df["Work Term Duration"])

In [49]:
df.head(3)

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,185577,Student Planner,City of St Catharines,Municipal Office,1,Open for Applications,St. Catharines,"Intermediate, Senior",12,19,1,Student Planner,To research assigned projects for data relevan...,Students should be experienced in virtual meet...,Targeted Clusters - Theme - Architecture and D...,True
1,186774,Student Planner,City of Waterloo,Development Services,1,Open for Applications,Waterloo,"Intermediate, Senior",63,19,1,The successful candidate will have involvement...,Growth Management Projects may include: Land u...,The candidate should have a strong working kno...,Targeted Clusters - Theme - Construction and I...,True
2,185901,Fleet Assistant,City of Waterloo,Divisional Office,1,Open for Applications,Waterloo,Junior,9,19,1,A co-op opportunity exists for a student to su...,"Provide support to the Fleet Coordinator, Flee...",Currently enrolled in a relevant post-secondar...,Targeted Clusters ARTS - Business,True


### Add a column for if the data is in the shortlist

In [59]:
esther_shortllist = pd.read_csv("scrape_shortlist.csv")
esther_shortllist = esther_shortllist.drop(['Unnamed: 0'], axis=1)
victor_shortlist = pd.read_csv("victor-shortlist.csv")
victor_shortlist = victor_shortlist.drop(['Unnamed: 10'], axis=1)

In [67]:
lists = [esther_shortllist, victor_shortlist]
sl = pd.concat(lists)
sl.drop_duplicates(subset=['ID'], inplace=True)
sl = sl.reset_index(drop=True)

In [69]:
sl[["ID", "Job Title", "Organization"]]

,ID,Job Title,Organization
0,187777,Data Science,Achievers Inc
1,183894,Engineering Co-op,ActiveState Software Inc
2,188750,Software Developer,AdHawk Microsystems
3,187277,Computer Vision Intern (Startup),Akasha Imaging
4,182773,Software Development Engineering - Summer 2021...,Amazon.com Inc
...,...,...,...
134,186745,NEW Data Science & Advanced Analytics,TD Bank Group
135,188232,NEW Junior Data Scientist,Triumf
136,188244,NEW Quantum Machine Learning Researcher,Triumf
137,187875,NEW Game Intelligence Analyst Intern,Ubisoft Toronto


In [70]:
df["would_shortlist"] = df["ID"].isin(sl["ID"])

In [6]:
# df = pd.read_csv('all_our_data_yay.csv')
# df = df.drop(['Unnamed: 0.1', 'Unnamed: 0', '3_in_one', 'text_final'], axis=1)
# df = df.fillna("")

In [7]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,would_shortlist
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False,False
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False,False
2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False,False
3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False,False
4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False,False
2478,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True,False
2479,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True,False
2480,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,3,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,True,False


In [8]:
 df["3_in_one"] = df.apply(lambda row: row["Job Summary"] + row["Job Responsibilities"] + row["Required Skills"], axis=1)

In [14]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, f1_score, recall_score
np.random.seed(500)
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split

In [10]:
# Change all the text to lower case
df["3_in_one"] = [entry.lower() for entry in df["3_in_one"]]
# Tokenization
df["3_in_one"]= [word_tokenize(entry) for entry in df["3_in_one"]]
# Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# We need Pos tags to understand if the word is noun or verb or whatever
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(df["3_in_one"]):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide if the word is noun or verb or whatever
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'text_final'] = str(Final_words)

In [79]:
df[df['would_shortlist'] == True]

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,3_in_one,text_final,would_shortlist
25,187777,Data Science,Achievers Inc,Divisional Office,2,Open for Applications,Toronto,"Intermediate, Senior",55,19,3,Data Science Co-Op Toronto Achievers delivers ...,"As a Data Science Co-op, you will: Design and ...",Qualifications: Enrolled in a Data Science or ...,Targeted Clusters - Theme - Computing: Softwar...,False,"['data', 'science', 'co-op', 'toronto', 'achie...","['data', 'science', 'toronto', 'achiever', 'de...",True
29,183894,Engineering Co-op,ActiveState Software Inc,Divisional Office,2,Open for Applications,Vancouver,"Intermediate, Senior",8,19,1,Engineering Internships (Co-Ops) ActiveState P...,What You'll be Doing As an intern on one of ou...,"Requirements Experience with Linux, Window or ...",Targeted Clusters - Theme - Computing: Informa...,True,"['engineering', 'internships', '(', 'co-ops', ...","['engineering', 'internship', 'activestate', '...",True
32,188750,Software Developer,AdHawk Microsystems,Divisional Office,1,Open for Applications,Kitchener,Senior,18,19,1,AdHawk Microsystems is changing the way that h...,AdHawk is looking for software developers to h...,- Strong software development fundamentals - E...,Targeted Clusters - Theme - Computing: Softwar...,False,"['adhawk', 'microsystems', 'is', 'changing', '...","['adhawk', 'microsystems', 'change', 'way', 'h...",True
51,187277,Computer Vision Intern (Startup),Akasha Imaging,Divisional Office,2,Open for Applications,Remote in Canada,"Intermediate, Senior",84,19,1,Akasha Imaging (http://akasha.im) is a VC-back...,- Independently implement algorithms for machi...,- Interest in understanding the fundamentals/p...,Targeted Clusters - Theme - Computing: Softwar...,False,"['akasha', 'imaging', '(', 'http', ':', '//aka...","['akasha', 'imaging', 'http', 'stealth', 'imag...",True
58,182773,Software Development Engineering - Summer 2021...,Amazon.com Inc,Head Office,1,Open for Applications,,"Intermediate, Senior",262,19,1,"At Amazon, we hire the best minds in technolog...",KEY JOB RESPONSIBILITIES · Collaborate with ex...,BASIC QUALIFICATIONS · Programming experience ...,Targeted Clusters - Theme - Computing: Informa...,False,"['at', 'amazon', ',', 'we', 'hire', 'the', 'be...","['amazon', 'hire', 'best', 'mind', 'technology...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2369,185126,Software Developer,Varicent,Development,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",36,19,1,Varicent is the leader in the Sales Performanc...,* Build new features for the REST API on top o...,* Knowledge and experience with RESTful APIs *...,Targeted Clusters - Theme - Computing: Softwar...,False,"['varicent', 'is', 'the', 'leader', 'in', 'the...","['varicent', 'leader', 'sale', 'performance', ...",True
2400,187337,"Associate Data Engineering - Platform, Infras...",Wattpad,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",26,19,1,Wattpad is a global multiplatform entertainmen...,- Evolving Wattpad's data infrastructure - Imp...,- Solid understanding of data engineering best...,Targeted Clusters - Theme - Computing: Softwar...,False,"['wattpad', 'is', 'a', 'global', 'multiplatfor...","['wattpad', 'global', 'multiplatform', 'entert...",True
2426,187923,Data Engineering,Wonolo Inc,Head Office,1,Open for Applications,Remote in Canada,"Intermediate, Senior",26,19,1,Wonolo (Work Now Locally) is growing and disru...,- Iterate on existing CNN models and in-house ...,- Strong comprehension of statistical analysis...,Targeted Clusters - Theme - Computing: Softwar...,False,"['wonolo', '(', 'work', 'now', 'locally', ')',...","['wonolo', 'work', 'locally', 'grow', 'disrupt...",True
2448,185105,Data Science Associate,XE.com,Divisional Office,1,Open for Applica

# Building the Model

In [16]:
x_data = df.drop('would_shortlist', axis=1)
y_data = df[['would_shortlist']]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.10, random_state=42)

In [19]:
Tfidf_vect = TfidfVectorizer(max_features=15000)
Tfidf_vect.fit(X_train["text_final"])
Train_X_Tfidf = Tfidf_vect.transform(X_train['text_final'])
Test_X_Tfidf = Tfidf_vect.transform(X_test['text_final'])

In [33]:
import itertools
import random
c_options = [0.2, 1, 4, 8]
kernal_options = ['linear', 'poly', 'rbf', 'sigmoid']
degree_options = [2,3,4] # go through only if poly
gamma_options = ['scale', 'auto'] # run only if not linear
coef0_options = [0,1,10,100] # run only poly sig
probability_options = [1,0]

In [37]:
my_combos = []
for c in c_options:
    for p in probability_options:
        for k in kernal_options:
            if k == "linear":
                my_combos += [(c, k, 1, 'auto', 0, p)]
            else:
                for g in gamma_options:
                    if k == "rbf":
                        my_combos += [(c, k, 1, g, 0, p)]
                    else:
                        for co in coef0_options:
                            if k == "sigmoid":
                                my_combos += [(c, k, 1, g, co, p)]
                            else:
                                for d in degree_options:
                                    my_combos += [(c, k, d, g, co, p)]
                                    
random.shuffle(my_combos)

In [40]:
best_combo_recall = {}

for c in my_combos[:100]:
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVM = svm.SVC(C=c[0], kernel=c[1], degree=c[2], gamma=c[3], coef0 = c[4], probability = c[5], class_weight = 'balanced')
    SVM.fit(Train_X_Tfidf,y_train['would_shortlist'])
    # predict the labels on validation dataset
    predictions_SVM = SVM.predict(Test_X_Tfidf)
    # Use accuracy_score function to get the accuracy
    my_recall_score = recall_score(predictions_SVM, y_test['would_shortlist'])*100
    #print("SVM Accuracy Score -> ",my_recall_score)
    best_combo_recall[c] = my_recall_score

C:\Users\esthe\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\esthe\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\esthe\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\esthe\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\_classifica

In [41]:
c = (8, 'poly', 2, 'scale', 1, 1)
SVM = svm.SVC(C=c[0], kernel=c[1], degree=c[2], gamma=c[3], coef0 = c[4], probability = c[5], class_weight = 'balanced')
SVM.fit(Train_X_Tfidf,y_train['would_shortlist'])

SVC(C=8, class_weight='balanced', coef0=1, degree=2, kernel='poly',
    probability=1)

# Making the Predictions

In [61]:
# import glob

# path = r'./2021' # use your path
# all_files = glob.glob(path + "/*.csv")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0)
#     li.append(df)

# df_to_predict = pd.concat(li, axis=0, ignore_index=True)


In [80]:
df_to_predict = pd.read_csv('cleaned_2021.csv')
df_to_predict = df_to_predict.drop(['Unnamed: 0'], axis=1)

In [58]:
import pickle
pickleFile = open("svm.pk", 'rb')
SVM = pickle.load(pickleFile)

In [82]:
To_Predict_Tfidf = Tfidf_vect.transform(df_to_predict['text_final'])
predictions_SVM = SVM.predict(To_Predict_Tfidf)
df_to_predict['yay'] = predictions_SVM

In [83]:
df_to_predict[df_to_predict['yay'] == True]

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,3_in_one,text_final,yay
35,221399.0,Engineering Co-op,ActiveState Software Inc,Divisional Office,2.0,Open for Applications,Vancouver,"Junior, Intermediate, Senior",11.0,"Sep 21, 2021",1,Engineering Internships (Co-Ops) ActiveState P...,What You'll Be Doing As an intern on one of ou...,"Requirements Experience with Linux, Window or ...",Targeted Clusters - Theme - Computing: Hardwar...,True,"['engineering', 'internships', '(', 'co-ops', ...","['engineering', 'internship', 'activestate', '...",True
43,220784.0,Embedded Software Developer,AdHawk Microsystems,Divisional Office,1.0,Open for Applications,Kitchener,Senior,46.0,"Sep 21, 2021",1,AdHawk Microsystems is changing the way that h...,AdHawk is looking for software developers to h...,- Strong software development fundamentals - S...,Targeted Clusters - Theme - Computing: Softwar...,False,"['adhawk', 'microsystems', 'is', 'changing', '...","['adhawk', 'microsystems', 'change', 'way', 'h...",True
44,220802.0,Full Stack Software Developer,AdHawk Microsystems,Divisional Office,1.0,Open for Applications,Kitchener,Senior,29.0,"Sep 21, 2021",1,AdHawk Microsystems is changing the way that h...,AdHawk is looking for software developers to h...,- Strong software development fundamentals - E...,Targeted Clusters - Theme - Computing: Softwar...,False,"['adhawk', 'microsystems', 'is', 'changing', '...","['adhawk', 'microsystems', 'change', 'way', 'h...",True
85,223272.0,Computer Vision or Software Engineering Intern,Akasha Imaging,Divisional Office,2.0,Open for Applications,Remote in Canada,"Intermediate, Senior",45.0,"Sep 21, 2021",1,Akasha Imaging (http://akasha.im) is a VC-back...,- Independently implement algorithms for machi...,- Interest in understanding the fundamentals/p...,Targeted Clusters - Theme - Computing: Softwar...,False,"['akasha', 'imaging', '(', 'http', ':', '//aka...","['akasha', 'imaging', 'http', 'imaging', 'comp...",True
95,218976.0,Associate ML Developer,AltaML Inc,Divisional Office,8.0,Open for Applications,Edmonton,Intermediate,49.0,"Sep 21, 2021",1,This opportunity is for current students and i...,What you'll do: Complete project related tasks...,"What you bring: Enrolled in BSc in Comp Sci, C...",Targeted Clusters - Theme - Computing: Informa...,False,"['this', 'opportunity', 'is', 'for', 'current'...","['opportunity', 'current', 'student', 'winter'...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651,220879.0,Data Science,ecobee,Divisional Office,1.0,Open for Applications,Toronto,"Intermediate, Senior",34.0,"Sep 21, 2021",1,Hi. We're ecobee. We introduced the world's fi...,How You'll Make an Impact: Participate in buil...,What You Bring to the Table: Maybe you're crea...,Targeted Clusters - Theme - Computing: Hardwar...,False,"['hi', '.', 'we', ""'re"", 'ecobee', '.', 'we', ...","['hi', 'ecobee', 'introduce', 'world', 'first'...",True
3699,222472.0,Algorithms / Machine Learning Engineering,Enlighted Inc,Waterloo,1.0,Open for Applications,Kitchener,"Intermediate, Senior",25.0,"Sep 21, 2021",1,Enlighted is a trailblazing Silicon Valley com...,"Design, develop and innovate enabled by our Io...",Very comfortable working in a Linux-based deve...,Targeted Clusters - Theme - Computing: Softwar...,False,"['enlighted', 'is', 'a', 'trailblazing', 'sili...","['enlighted', 'trailblazing', 'silicon', 'vall...",True
3727,223607.0,Junior Environmental Research Scientist,Environment and Climate Change Canada,Meteorological Research Division,1.0,Open for Applications,NaN,"Intermediate, Senior",9.0,"Sep 21, 2021",1,The Meteorological Research Division of Enviro...,"- Import, ingest and visualize meteorological ...",- Mathematics and statistic prowess - Computer...,Targeted Clusters - Theme - Computing: Softwar...,False,"['

In [84]:
df_to_predict[df_to_predict['Organization'] == 'Stratum AI']

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,3_in_one,text_final,yay
2470,223631.0,Data Scientist/ML Ops Engineering,Stratum AI,Divisional Office,2.0,Open for Applications,Richmond Hill,"Intermediate, Senior",9.0,"Sep 21, 2021",1,"Here at StratumAI, we work towards improving t...",Responsibilities Developing and implementing d...,Required Skills: Experience working with machi...,Targeted Clusters - Theme - Computing: Informa...,False,"['here', 'at', 'stratumai', ',', 'we', 'work',...","['stratumai', 'work', 'towards', 'improve', 'm...",True
